<a href="https://colab.research.google.com/github/drfperez/psychometry/blob/main/CCBBpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Instal·lació de dependències (executa-ho un cop al teu Colab)
!pip install pingouin

import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr
import pingouin as pg
from google.colab import files
import io

# 1) Puja el CSV
print("Puja el teu fitxer CSV:")
uploaded = files.upload()

# 2) Llegeix el CSV en un DataFrame
fname = list(uploaded.keys())[0]
df_raw = pd.read_csv(io.BytesIO(uploaded[fname]))

# Si hi ha una columna d'identificadors no numèrica, la fem fora:
df = df_raw.select_dtypes(include=[np.number]).copy()

# Forcem valors enters 0/1 i eliminem NA
df = df.fillna(0).astype(int)

# 3) Càlcul de dificultat (mitjana per ítem)
difficulty = df.mean()

# Total de puntuació per participant
total_score = df.sum(axis=1)

# 3) Càlcul de discriminació i punt‐biserial
discrimination = {}
pbis = {}
for col in df.columns:
    item = df[col]
    rest = total_score - item
    discrimination[col] = item.corr(rest)
    r_pb, _ = pointbiserialr(item, total_score)
    pbis[col] = r_pb

# 4) Alfa de Cronbach
alpha, _ = pg.cronbach_alpha(df)

# 5) Taula de resultats
results = pd.DataFrame({
    'Item': df.columns,
    'Dificultat': difficulty.values,
    'Discriminació': [discrimination[c] for c in df.columns],
    'Punt‐Biserial': [pbis[c] for c in df.columns]
})

# Ítems problemàtics
problematic = results[
    (results['Discriminació'] < 0.2) |
    (results['Punt‐Biserial'] < 0.2)
]['Item'].tolist()

# --- Sortida per pantalla ---
print("\n╔══════════ ANÀLISI PSICOMÈTRIC ══════════╗")
print(results.to_string(index=False))
print("╚═══════════════════════════════════════╝")

print(f"\nAlfa de Cronbach: {alpha:.3f}")

if problematic:
    print("\nÍtems problemàtics (Discriminació o Punt‐Biserial < 0.2):")
    for it in problematic:
        print(" -", it)
else:
    print("\nNo s'han detectat ítems problemàtics.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 2.1 MB/s eta 0:00:00
Puja el teu fitxer CSV:


Saving dades (13).csv to dades (13).csv

╔══════════ ANÀLISI PSICOMÈTRIC ══════════╗
 Item  Dificultat  Discriminació  Punt‐Biserial
   Q1        0.71      -0.053564       0.083438
   Q2        0.74      -0.003745       0.128614
   Q3        0.71       0.108740       0.243002
   Q4        0.69      -0.090663       0.048596
 Q5_1        0.72       0.050806       0.185445
 Q5_2        0.67      -0.069053       0.072820
 Q5_3        0.65      -0.015942       0.128081
 Q5_4        0.73      -0.007261       0.126732
   Q6        0.72      -0.064264       0.071222
   Q7        0.71       0.115664       0.249650
   Q8        0.73       0.061816       0.194685
 Q9_1        0.71      -0.060171       0.076790
 Q9_2        0.67      -0.088066       0.053573
 Q9_3        0.79       0.272973       0.384780
 Q9_4        0.57       0.070636       0.218458
  Q10        0.76       0.025442       0.153991
  Q11        0.67       0.080250       0.220385
  Q12        0.65       0.107115       0.248256
Q13